# Experiment 3: Time-Aware Baseline Model
Using only hour of day and station ID.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from capymoa.models import FIMTDDRegressor
from capymoa.stream import DataStream

In [ ]:
# Load dataset
df = pd.read_csv('MoreModels/loader_f30avg.csv')
df['datetime'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['datetime'].dt.hour
df['station'] = df['station_id'].astype(str)

In [ ]:
# Prepare features and target
features = df[['hour', 'station']]
target = df['target']

encoder = OneHotEncoder(sparse=False)
X_encoded = encoder.fit_transform(features)

X_array = X_encoded.astype(np.float32)
y_array = target.to_numpy().astype(np.float32)

stream = DataStream(X_array, y_array)

In [ ]:
# Initialize model and delayed evaluation
model = FIMTDDRegressor()
true_values = []
predictions = []
buffer_X = []
buffer_y = []

for x_i, y_i in stream:
    if len(buffer_X) >= 6:
        pred = model.predict([buffer_X.pop(0)])[0]
        true = buffer_y.pop(0)
        predictions.append(pred)
        true_values.append(true)

    model.partial_fit([x_i], [y_i])
    buffer_X.append(x_i)
    buffer_y.append(y_i)

In [ ]:
# Results
mae = mean_absolute_error(true_values, predictions)
rmse = sqrt(mean_squared_error(true_values, predictions))

print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")